In [43]:
import tkinter as tk
from tkinter import messagebox
import pandas as pd


In [44]:
try:
    df = pd.read_csv("record.csv")
except Exception as e:
    data = {
        "Date": [],
        "Description": [],
        'Amount': [],
        "Type":[]
    }
        
    df = pd.DataFrame(data)


In [49]:
class BudgetTrackerApp:
    def __init__(self,root):
        self.root = root
        self.root.title("Budget Tracker")

        #Labels and entries for input

        #Date input
        self.label_date = tk.Label(root,text="Date (YY-MM-DD):")
        self.label_date.grid(row=0,column=0)
        self.entry_date = tk.Entry(root)
        self.entry_date.grid(row=0,column=1)

        #Description input
        self.label_description = tk.Label(root,text="Description:")
        self.label_description.grid(row=1,column=0)
        self.entry_description = tk.Entry(root)
        self.entry_description.grid(row=1,column=1)

        #amount input
        self.label_amount = tk.Label(root,text="Amount:")
        self.label_amount.grid(row=2,column=0)
        self.entry_amount = tk.Entry(root)
        self.entry_amount.grid(row=2,column=1)


        self.label_type = tk.Label(root,text="Type (Income/expense):")
        self.label_type.grid(row=3,column=0)
        self.entry_type = tk.Entry(root)
        self.entry_type.grid(row=3,column=1)

        #create buttons
        self.button_add = tk.Button(root,text="Add entry",command=self.add_entry)
        self.button_add.grid(row=4,column=0,columnspan=2)

        self.button_view = tk.Button(root,text="View Entries",command=self.view_entries)
        self.button_view.grid(row=5,column=0,columnspan=2)


    def add_entry(self):
        date = self.entry_date.get() 
        description = self.entry_description.get()
        amount = self.entry_amount.get() 
        entry_type = self.entry_type.get()

        if date and description and amount and entry_type:
            new_entry = pd.DataFrame( data = {
            "Date": [date],
            "Description": [description],
            'Amount': [amount],
            "Type":[entry_type]
            })
            global df
            df = pd.concat([df,new_entry],ignore_index=True)
            df.to_csv("record.csv",index=False)
            messagebox.showinfo("Success","Entry saved successfully")
        else:
            messagebox.showerror("Error","All fields are required!")    




    def view_entries(self):
        global df
        top = tk.Toplevel(self.root)
        top.title("View entries")
        text = tk.Text(top)
        text.pack()

        for index,row in df.iterrows():
            text.insert(tk.END,"Date:"+ row['Date'] + "| Description:" + row["Description"] + "|  Amount:" + str(row["Amount"]) + "|  Type:" + row["Type"] + "\n")


In [50]:
root = tk.Tk()
app = BudgetTrackerApp(root)
root.mainloop()

In [ ]:
import math